In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('hw') \
    .getOrCreate()
spark.version

In [ ]:
!cd ../data/hw/ && curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [ ]:
df = spark.read.parquet('../data/hw/yellow_tripdata_2024-10.parquet')

In [ ]:
df = df.repartition(4)
df.write.parquet('../data/hw/pq/2024/10/', mode='overwrite')

In [ ]:
!ls -lh ../data/hw/pq/2024/10/

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df.filter(
    F.to_date('tpep_pickup_datetime') == '2024-10-15'
).count()

In [ ]:
df.withColumn(
    'time_diff_hrs', 
    (F.unix_timestamp('tpep_dropoff_datetime') - F.unix_timestamp('tpep_pickup_datetime')) / 3600
).select(
    F.max('time_diff_hrs')
).show()

In [ ]:
zones_df = spark.read \
    .option('header', 'true') \
    .csv('../data/tst/taxi_zone_lookup.csv')

In [ ]:
zones_df.columns

In [ ]:
zones_df.createOrReplaceTempView('zones')
df.createOrReplaceTempView('trips')

In [ ]:
df.columns

In [ ]:
spark.sql(
    """
    SELECT
        pickup_zone.zone as pickup_zone,
        COUNT(1) AS zone_pickups_cnt
    FROM trips
        INNER JOIN zones AS pickup_zone
            ON trips.PULocationID = pickup_zone.LocationID
        INNER JOIN zones AS dropoff_zone
            ON trips.DOLocationID = dropoff_zone.LocationID
    GROUP BY pickup_zone
    ORDER BY zone_pickups_cnt
    """
).show()